In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

In [ ]:
data = pd.read_csv('/content/university_classification.csv')

sentences = data['pertanyaan']
university = data['universitas']

# Membersihkan data dari nilai NaN
university = university.dropna()

training_sentences, testing_sentences, train_university, test_university = train_test_split(sentences, university, test_size = 0.1, stratify = university)

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)

train_sequences = tokenizer.texts_to_sequences(training_sentences)
train_padded = pad_sequences(train_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)

validation_sequences = tokenizer.texts_to_sequences(testing_sentences)
valid_padded = pad_sequences(validation_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(university)

train_label_encoded = label_encoder.transform(train_university)
test_label_encoded = label_encoder.transform(test_university)

num_classes = len(label_encoder.classes_)

train_label_final = to_categorical(train_label_encoded, num_classes=num_classes)
test_label_final = to_categorical(test_label_encoded, num_classes=num_classes)

In [ ]:
# encoder_url = 'https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/multi-cased-l-12-h-768-a-12/versions/4'
# preprocess_url = "https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/multi-cased-preprocess/versions/3"
# embedding_model1 = hub.KerasLayer('encoder_url', trainable=True)
# preprocess_model1 = hub.KerasLayer(preprocess_url)

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = None):
    if logs.get('val_loss') < 0.60 and logs.get('val_accuracy') >= 0.85:
      self.model.stop_training = True

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(20, activation='softmax')
])

In [ ]:
model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)

In [ ]:
model.fit(
    train_padded,
    train_label_final,
    validation_data=(valid_padded, test_label_final),
    epochs=100,
    batch_size=16,
    callbacks=[MyCallback()]
)

Epoch 1/100
27/27 [==============================] - 20s 211ms/step - loss: 2.9988 - accuracy: 0.0509 - val_loss: 2.9975 - val_accuracy: 0.0417
Epoch 2/100
27/27 [==============================] - 5s 174ms/step - loss: 2.9990 - accuracy: 0.0231 - val_loss: 2.9961 - val_accuracy: 0.0625
Epoch 3/100
27/27 [==============================] - 6s 210ms/step - loss: 2.9977 - accuracy: 0.0509 - val_loss: 2.9975 - val_accuracy: 0.0417
Epoch 4/100
27/27 [==============================] - 4s 144ms/step - loss: 2.9972 - accuracy: 0.0556 - val_loss: 2.9984 - val_accuracy: 0.0625
Epoch 5/100
27/27 [==============================] - 4s 143ms/step - loss: 2.9921 - accuracy: 0.0625 - val_loss: 2.9828 - val_accuracy: 0.0833
Epoch 6/100
27/27 [==============================] - 6s 218ms/step - loss: 2.8367 - accuracy: 0.0995 - val_loss: 2.6602 - val_accuracy: 0.0833
Epoch 7/100
27/27 [==============================] - 4s 162ms/step - loss: 2.5223 - accuracy: 0.1181 - val_loss: 2.4222 - val_accuracy: 0.250

In [ ]:
# Evaluasi model menggunakan classification report
from sklearn.metrics import classification_report

predictions = model.predict(valid_padded)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_label_final, axis=1)
class_labels = label_encoder.classes_

print(classification_report(true_classes, predicted_classes, target_names=class_labels))

2/2 [==============================] - 2s 40ms/step
                                     precision    recall  f1-score   support

           Institut Pertanian Bogor       1.00      1.00      1.00         2
         Institut Teknologi Bandung       1.00      1.00      1.00         3
Institut Teknologi Sepuluh Nopember       1.00      0.33      0.50         3
              Universitas Airlangga       0.50      0.67      0.57         3
               Universitas Atmajaya       1.00      0.67      0.80         3
         Universitas Bina Nusantara       1.00      1.00      1.00         3
              Universitas Brawijaya       0.50      1.00      0.67         2
             Universitas Diponegoro       1.00      1.00      1.00         2
            Universitas Gadjah Mada       1.00      1.00      1.00         2
             Universitas Hasanuddin       0.00      0.00      0.00         2
              Universitas Indonesia       1.00      1.00      1.00         3
        Universitas Isl

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/model_university_classification')

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/model_university_classification')

In [ ]:
import numpy as np

# Input teks yang ingin Anda uji
input_text = ['Kapan Universitas Udayana didirikan?']

# Tokenisasi dan padding input
input_sequences = tokenizer.texts_to_sequences(input_text)
padded_input = pad_sequences(input_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Prediksi dengan model
predictions = model.predict(np.array(padded_input))  # Konversi list menjadi array NumPy

# Temukan indeks dengan nilai tertinggi
predicted_class_index = np.argmax(predictions)

# Dapatkan daftar kelas yang telah diencode
classes = label_encoder.classes_

# Konversi indeks ke label kelas
predicted_class_label = classes[predicted_class_index]

# Tampilkan hasil prediksi
print(f"Predicted class: {predicted_class_label}")

1/1 [==============================] - 0s 85ms/step
Predicted class: Universitas Udayana


In [ ]:
model.save_weights('/content/drive/MyDrive/Colab Notebooks/university_classification.h5')